In [13]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 

In [95]:
# initial variables
search_term = "hoodies for men"
pages = 20
records = []
template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
search_text = search_term.replace(' ', '+')
# add term query to url
url = template.format(search_text)
# add page query placeholder
url += '&page={}'

print(url)

https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page={}


In [96]:
# startup the webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())

for page in range(1, pages+1):
    formated_url = url.format(page)
    print(f"Scraping from url {formated_url} page {page}")
    
    driver.get(formated_url)
    driver.implicitly_wait(5) # gives an implicit wait for 5 seconds
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})
    
    for item in results:
        # description and url
        atag = item.h2.a
        description = atag.text.strip()
        item_url = 'https://www.amazon.com' + atag.get('href')
        
        try:
            # product price
            price_parent = item.find('span', 'a-price')
            price = price_parent.find('span', 'a-offscreen').text
            # rating and review count html 
            rating = item.find('span', {'class': 'a-icon-alt'}).text[0:3]
            review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
        except AttributeError:
            rating = ''
            review_count = ''
            
        record = (description, price, rating, review_count, item_url)
        if record:
            records.append(record)

driver.close()

print(f"\nScraped. total {len(records)+1} records")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Pleum\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=1 page 1
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=2 page 2
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=3 page 3
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=4 page 4
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=5 page 5
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=6 page 6
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=7 page 7
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=8 page 8
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=9 page 9
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&page=10 page 10
Scraping from url https://www.amazon.com/s?k=hoodies+for+men&ref=nb_sb_noss_1&

In [99]:
# show records
df = pd.DataFrame(records)
df.columns = ['Description', 'Price', 'Rating', 'ReviewCount', 'Url']

df

,Description,Price,Rating,ReviewCount,Url
0,Mens Hoodies Pullover Color Block Fleece Long ...,$29.99,4.4,666,https://www.amazon.com/gp/slredirect/picassoRe...
1,"Gildan Adult Fleece Hooded Sweatshirt, Style G...",$11.85,4.6,"92,168",https://www.amazon.com/gp/slredirect/picassoRe...
2,Gesean Men's Novelty Color Block Pullover Flee...,$26.99,4.5,"6,713",https://www.amazon.com/gp/slredirect/picassoRe...
3,LAIWANG Men's Color Block Hoodie-Full Zip Ulti...,$39.99,4.5,"1,204",https://www.amazon.com/gp/slredirect/picassoRe...
4,Hanes Men's Pullover EcoSmart Hooded Sweatshirt,$15.00,4.5,"141,861",https://www.amazon.com/Hanes-Pullover-EcoSmart...
...,...,...,...,...,...
489,Gesean Men's Novelty Color Block Pullover Flee...,$29.99,4.5,"6,713",https://www.amazon.com/gp/slredirect/picassoRe...
490,Mens Long Sleeve Full-Zip Up 3D Novelty Hoodie...,$17.95,3.9,15,https://www.amazon.com/gp/slredirect/picassoRe...
491,Neleus Men's Running Shirt Long Sleeve Workout...,$28.98,4.3,571,https://www.amazon.com/gp/slredirect/picassoRe...
492,TACVASEN Men's Hiking Shirts UPF 50+ UV Sun Pr...,$21.98,4.3,187,https://www.amazon.com/gp/slredirect/picassoRe...


In [98]:
# save data to csv file
search_term = search_term.replace(' ', '_')

from datetime import datetime
cur_date = datetime.today().strftime('%d-%m-%Y')

with open(f'{search_term} {cur_date}.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
    writer.writerows(records)
    
print('finish')

finish
